In [249]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np

In [292]:
#Der Header funkt
HEADERS = {'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'}
#Die Keywords, für die die Results page angezeigt werden soll
Keywords=["Laptop Ständer"]
#Titel des Ref. Produkts (muss noch angepasst werden auf Liste von Links..)
Ref_prod="Laptop ständer, Einstellbar Notebook Ständer, Aluminium Faltbareres Laptopständer, kompatibel für Laptop (10-17 Zoll), MacBook Pro/Air, Dell, Lenovo (Silber)"

In [41]:
#Soups der Result pages
Result_soups = [BeautifulSoup(requests.get("https://www.amazon.de/s?k="+keyw.replace(" ", "+"), headers=HEADERS).text, "html5lib") for keyw in Keywords]

In [206]:
#Gibt Listen der ASINs und Titel der Produkte wieder
def Get_items(soups):
    urls=[]
    Titles=[]
    ASINs=[]
    for soup in soups:
        for Prod_q in soup.select("a.a-link-normal.a-text-normal"):
            Prod=Prod_q.select('span.a-size-base-plus.a-color-base.a-text-normal')
            if Prod!=[]:
                Titles.append(Prod[0].get_text())
                hreff=re.search("(?<=dp).*", Prod_q.find_parent().find(name="a")["href"]).group(0)
                if hreff[0]=="%":
                    ASINs.append(hreff.split("%")[1][2:])
                else:
                    ASINs.append(hreff.split("/")[1])
    return Titles, ASINs

In [277]:
#Listen der Titel und ASINs 
Titles, ASINs=Get_items(Result_soups)
#Produkt-Urls
urlss=["https://www.amazon.de/dp/"+ ASIN for ASIN in ASINs]

Laptop ständer, Einstellbar Notebook Ständer, Aluminium Faltbareres Laptopständer, kompatibel für Laptop (10-17 Zoll), MacBook Pro/Air, Dell, Lenovo (Silber)


In [297]:
#Provisorische Blacklist. Elemente aus dieser Liste werden aus den Titeln entfernt.
Clean_prov=[",", "/", "(", ")"]

#Entfernt Strings die in "Clean_prov" enthalten sind aus den Titeln
#Und Gibt Listen der übrigen Wörter wieder: "Titel(,) Test" -> ["Titel", "Test"]
def Clean_wordlister(Title, Clean=Clean_prov):
    for elem in Clean:
        Title=Title.replace(elem, "")
    return list(dict.fromkeys(Title.split(" ")))

#Wendet Clean_wordlister auf den Referenztitel und die Vergleichstitel
#der Results an und speichert diese Listen in Clean_ref & Clean_Titles
Clean_ref=Clean_wordlister(Ref_prod)
Clean_Titles=[]
for elem in Titles:
    Title=Clean_wordlister(elem)
    Clean_Titles.append(Title)
    
#Für jedes Wort im Titel des Referenzprodukts das im zum vergleichenden Produkttitel vorkommt
#+1 Score, Geteilt durch die Anzahl der Wärter
def Assess_products(titles=Clean_Titles, ref=Clean_ref):
    Scores=np.zeros((len(titles)))
    for elem in ref:
        for i in range(len(titles)):
            for j in range(len(titles[i])):
                if elem==titles[i][j]:
                    Scores[i]+=1
    return [Scores[i]/len(Clean_Titles[i]) for i in range(len(Clean_Titles))]
print(Assess_products())

['Laptop', 'ständer', 'Einstellbar', 'Notebook', 'Ständer', 'Aluminium', 'Faltbareres', 'Laptopständer', 'kompatibel', 'für', '10-17', 'Zoll', 'MacBook', 'ProAir', 'Dell', 'Lenovo', 'Silber']
[0.375, 1.0, 0.45, 0.47619047619047616, 0.35, 0.2608695652173913, 0.32142857142857145, 0.2, 0.2608695652173913, 0.25, 0.0, 0.375, 0.2727272727272727, 0.1, 0.07692307692307693, 0.2222222222222222, 0.2608695652173913, 0.3333333333333333, 0.2608695652173913, 0.3333333333333333, 1.0, 0.47619047619047616, 0.36363636363636365, 0.30434782608695654, 0.43478260869565216, 0.2631578947368421, 0.47368421052631576, 0.3333333333333333, 0.2727272727272727, 0.23076923076923078, 0.2727272727272727, 0.375, 0.4583333333333333, 0.391304347826087, 0.07142857142857142, 0.125, 0.2727272727272727, 0.07142857142857142, 0.25, 0.16666666666666666, 0.3181818181818182, 0.34782608695652173, 0.47058823529411764, 0.2727272727272727, 0.1111111111111111, 0.3333333333333333, 0.16666666666666666, 0.4, 0.45454545454545453, 0.47368421